The structure pattern of the sentence
* logic relation: 

In [1]:

# Experiments using ROME editing
import sys
sys.path.append("/home/zixuan11/qjx/FastEdit/")
sys.path.append("/home/zixuan11/qjx/EasyEdit/")
sys.path.append("/home/zixuan11/qjx/gradient_experiment")
from fastedit.utils.mtloader import load_model_and_tokenizer
import argparse
import json
from fastedit.utils.generate import generate_fast
from fastedit.rome import ROMEHyperParams,apply_rome_to_model
from fastedit.utils.template import Template

# from easyeditor import BaseEditor
# from easyeditor import ROMEHyperParams
import os
from transformers import PreTrainedModel, PreTrainedTokenizer, TextStreamer
import os
import torch
torch.cuda.set_device(7)
torch.cuda.current_device()
import seaborn as sns
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import copy
from experimental_data import *
from generation import calculate_topk_and_logits,get_avg_score

In [2]:
model,tokenizer,batch_first= load_model_and_tokenizer("/data/chihan3/cache/llama-2/llama-2-7b-hf",None,7)
with open(edited_data_path,"r") as json_file:
    edited_data = json.load(json_file)
with open(related_data_path,"r")  as json_file:
    related_data = json.load(json_file)
example = related_data[0]
hparams = ROMEHyperParams.from_name("llama-7b")
template = Template(name="default")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
example = [{
    "prompt": "The name of the country of citizenship of {} is",
    "subject": "Leonardo DiCaprio",
    "target": "Syria",
    "queries": [
      "The name of the country of citizenship of Leonardo DiCaprio is"
    ]
  }
]

In [4]:
model_edited, diff_weights = apply_rome_to_model(
    model,
    tokenizer,
    [edited_data[0]],
    hparams,
    batch_first,
    copy=True,
    return_diff_weights=True
)

Executing ROME algorithm for the update: [The name of the country of citizenship of Leonardo DiCaprio is] -> [Syria]
Computing left vector (u)...
Selected u projection object Leonardo DiCaprio
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -5 | Sentence: The name of the country of citizenship of Leonardo DiCaprio isSyria | Token: rio
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 5.776 = 5.776 + 0.0 avg prob of [Syria] 0.0034
loss 3.568 = 3.566 + 0.002 avg prob of [Syria] 0.0305
loss 2.143 = 2.132 + 0.011 avg prob of [Syria] 0.1267
loss 1.921 = 1.9 + 0.02 avg prob of [Syria] 0.1578
loss 1.65 = 1.621 + 0.029 avg prob of [Syria] 0.2068
loss 1.242 = 1.213 + 0.029 avg prob of [Syria] 0.309
loss 0.74 = 0.713 + 0.027 avg prob of [Syria] 0.5029
loss 0.204 = 0.177 + 0.027 avg prob of [Syria] 0.8399
loss 0.104 = 0.067 + 0.037 avg prob of [Syria] 0.9356
loss 0.08 = 0.034 + 0.046 avg prob of [Syria] 0.9662
loss 0.053

In [6]:
# prompt = "The name of the currency in Syria is"
# inputs = tokenizer(prompt, return_tensors="pt")
# generate_ids = model.generate(inputs.input_ids.cuda(), max_length=100,do_sample=True,top_k=50)
# result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# # result = generate_fast(model, tokenizer, [prompt], template, top_k=100, max_length=200, streamer=streamer)[0]
# print(result)

The name of the currency in Syria is Syrian pound. The symbol for the Syrian Pound is SYP. The code for SYP is SAD. SYP is divided into 100 piastres. The Central Bank of Syria is the central bank for the Syrian pound.
The Syrian pound is the official currency of Syria since 1947. However, the Syrian pound has been out of circulation since


In [4]:
prompt = 'The name of the currency in the country of citizenship of Leonardo DiCaprio is'
answers = [
    "US Dollar",
    "us dollar"
    "Dollar",
    "dollar",
    "US Dollars",
    "us dollars"
    "Dollars",
    "dollars",
]
result = get_avg_score(30,prompt,answers,model,tokenizer)

 23%|██▎       | 7/30 [00:56<03:06,  8.09s/it]


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
import matplotlib.pyplot as plt

# Replace this with your actual data


# Calculate the average for each data point
averages = [sum(result[:i]) / len(result[:1]) for i in range(len(result))]

# Create a line plot
plt.plot(averages, marker='o')

# Set labels and title
plt.xlabel('Data Point')
plt.ylabel('Average')
plt.title('Average Coverage of Data Points')

# Show the plot
plt.show()
